# Guía práctica: Creación y configuración de clusters (Compute) en Databricks – AWS
**Fecha de preparación:** 2025-10-16

Este notebook explica los campos y decisiones al crear un **cluster (compute)** en Databricks sobre AWS, incluye ejemplos prácticos, casos de uso y una **configuración recomendada para el curso**.

---
## Objetivos
- Entender cada parámetro del formulario **Create new compute**.
- Conocer tipos de clúster (All-purpose vs Job, Single node vs Multi-node) y modos de acceso.
- Aplicar buenas prácticas de rendimiento, seguridad y costos.
- Verificar la configuración desde código (Spark/Python/SQL).


## Conceptos esenciales
**Compute / Cluster**: Conjunto de instancias gestionadas por Databricks para ejecutar notebooks y jobs.

**All-purpose cluster**: Para desarrollo interactivo en notebooks; varios usuarios pueden adjuntarse.

**Job cluster**: Se crea/derriba por ejecución de un Job; aislamiento y control de costos.

**Single node**: Un solo nodo (driver); útil para desarrollo ligero, bibliotecas de Python o pequeños datasets.

**Multi-node**: Un driver + N workers para procesamiento distribuido.

**Databricks Runtime (DBR)**: Imagen preconfigurada con Spark, Delta y librerías. Puede ser **LTS** (soporte extendido) o con componentes de ML.

**Photon**: Motor vectorizado que acelera cargas SQL y Delta Lake.

**Autoscaling**: Ajusta dinámicamente el número de workers entre un mínimo y un máximo.

**Auto-termination**: Apaga el clúster tras N minutos de inactividad para ahorrar costos.

**Unity Catalog (Access mode)**: Gobierno centralizado de datos y permisos a nivel de objeto.


## Campos del formulario "Create new compute" y su significado
1. **Compute name**: Nombre descriptivo (conviene incluir propósito y entorno, p. ej. `course-dev-ap`).
2. **Policy**: Plantilla que restringe/estandariza opciones (si el admin la define). Para el curso, suele ser `Unrestricted`.
3. **Machine learning**: Habilita runtimes con librerías de ML preinstaladas (scikit-learn, MLlib extra, etc.). Útil si entrenará modelos en el notebook.
4. **Databricks runtime**: Versión del DBR. Preferir **LTS** estable. Opcional: *Photon acceleration* para SQL/Delta.
5. **Worker type**: Tipo/tamaño de instancia de AWS para los workers (CPU, memoria, red). Seleccione equilibrado.
6. **Min / Max**: Rango de autoscaling. Para cargas pequeñas, mantenga valores modestos.
7. **Single node**: Forzar clúster de un solo nodo (driver). Útil para desarrollo/local libraries.
8. **Enable autoscaling**: Marca para activar escalado automático.
9. **Terminate after**: Minutos de inactividad antes de apagar (control de costos).
10. **Advanced performance**: Opciones adicionales del runtime (Delta caching, etc.) según versión.
11. **Tags**: Etiquetas de costo/propiedad (clave-valor) propagadas a AWS.
12. **Access mode** (Unity Catalog): `Auto` o `Manual` para determinar modo de acceso y aislamiento UC.
13. **Instance profile**: Rol/credencial de IAM para acceder a S3 u otros servicios (si requiere datos externos).
14. **Spark** (config): Clave-valor para *tuning* (p. ej., `spark.sql.shuffle.partitions`).
15. **SSH**: Habilitar acceso SSH al driver (solo si es necesario y permitido por seguridad).
16. **Logging**: Envío de logs a S3/DBFS para auditoría.
17. **Init scripts**: Scripts de inicialización para instalar dependencias o configurar el entorno.


## Modos de acceso y Unity Catalog
- **Auto/UC**: Aísla identidades y permisos por usuario/tablas; recomendado si su workspace usa Unity Catalog.
- **Shared/Single User/No Isolation** (según disponibilidad de la versión): definen el aislamiento de credenciales y cómo se resuelven permisos de datos.
Para el curso, use **Access mode: Auto** salvo que requiera un modo específico por políticas.


## Tipos de clúster y casos de uso
### All-purpose vs Job
- **All-purpose**: Desarrollo colaborativo, EDA, pruebas iterativas.
- **Job**: Producción/orquestación con Workflows; crea/termina por ejecución, cost-efficient y reproducible.

### Single node vs Multi-node
- **Single node**: Notebooks de utilería, bibliotecas Python, datasets pequeños (< 2–5 GB) o pruebas rápidas.
- **Multi-node**: ETL distribuidas, uniones/agrupaciones grandes, entrenamiento ML distribuido.


## Buenas prácticas de rendimiento
- Activar **Photon** en cargas SQL/Delta intensivas.
- Ajustar `spark.sql.shuffle.partitions` a un valor acorde al tamaño (p. ej., 64 para datasets pequeños del curso).
- Usar **broadcast joins** cuando una tabla < tamaño de broadcast (`spark.sql.autoBroadcastJoinThreshold`).
- **Compaction** y **Z-Order** para Delta tras cargas grandes.
- **Cache** selectivo (Delta caching / `df.cache()`) cuando haya reutilización de resultados.


## Seguridad y acceso a datos
- Preferir **Unity Catalog** para permisos de objetos (catalogs/schemas/tables).
- Para acceso a S3, usar **Instance Profiles** o **External Locations**; evitar llaves estáticas.
- Principio de **mínimo privilegio** en IAM.
- Registrar logs del clúster y auditar accesos.


## Costos: recomendaciones
- **Auto-termination** entre 30 y 90 minutos en ambientes de entrenamiento.
- **Autoscaling** con *Min* bajo y *Max* razonable.
- Elegir **instancias equilibradas** (vCPU/memoria) y evitar sobreaprovisionamiento.
- Limpiar clústeres inactivos y usar **Job clusters** para pipelines programados.


## Configuración recomendada para el curso (AWS Premium Trial)
- **Databricks Runtime**: `16.4 LTS` (Spark 3.5.x) con **Photon** habilitado.
- **Tipo de clúster**: **All-purpose**, **Multi-node**.
- **Worker type**: tamaño equilibrado (por ejemplo, `r5d.xlarge` o el equivalente gestionado por *fleet* con ~4 vCPU y ~32 GB RAM).
- **Autoscaling**: **Min 2**, **Max 4** workers.
- **Auto-termination**: **60 minutos**.
- **Access mode**: **Auto** (Unity Catalog).
- **Instance profile**: `None` (a menos que necesite acceso a buckets S3 externos específicos).
- **Spark config** (ejemplo):
  - `spark.sql.shuffle.partitions=64`
  - `spark.databricks.delta.optimizeWrite.enabled=true`
  - `spark.databricks.delta.autoCompact.enabled=true`


## Verificación desde código (ejemplos)
Use estas celdas para confirmar versión de Spark, Photon, y parámetros clave del clúster.


In [ ]:

# TODO: Versión y propiedades básicas de Spark

In [ ]:

# TODO: Comprobar configuraciones relevantes

In [ ]:

# TODO:  Ajustar particiones de shuffle para el contexto del curso (si no está fijado por política)

### SQL: exploración rápida de catálogos y tablas
Ejemplo de consultas que suelen funcionar en workspaces con `samples`.


### Carga de ejemplo (distribuida)
La siguiente celda crea un DataFrame de ejemplo y realiza una agregación con *shuffle* para observar el comportamiento del clúster.


In [ ]:

# TODO: Ejemplo de carga distribuida

## Troubleshooting habitual
- **Cluster no arranca**: revisar cuota de instancias en AWS y políticas; intente un tipo de instancia diferente.
- **Permisos a datos denegados**: validar Access mode de UC, grants en objetos y/o Instance Profile.
- **Jobs lentos**: revisar *shuffle partitions*, tamaño de clúster, Photon y optimizaciones Delta.
- **Fallas por librerías**: usar `Init scripts` o *Libraries* del clúster; fijar versiones compatibles.


## Conclusiones
- Elegir el **runtime LTS** y habilitar **Photon** acelera la mayoría de workloads analíticas.
- Diseñar el clúster con **autoscaling** y **auto-termination** controla costos en el trial.
- Validar y ajustar parámetros de Spark desde el notebook mejora el rendimiento en datasets pequeños del curso.
- Para producción, preferir **Job clusters** orquestados con Workflows y *policies* de clúster.
